In [1]:
import torch
import numpy as np
from graph.matrixview import render_incidence
from engine.torchengine import AnalyticalSetSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, FunctionSympy,get_analysis_structure, ElimResidual, ParallelResiduals, ParallelAnalysis, ipoptsolvercon
from engine.torchdata import load_file, process_expression, load_vals, generate_optim_functions, print_formatted_table, perturb
from engine.loaddata import process_json
import json
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
data = load_file('testproblems')
problem_id = 0
_, fdata = list(data.items())[problem_id]
polynomials, indices, edges, objective = process_json(fdata)
idxrev = {var.item():key for key,var in indices.items()}

In [3]:
def runproblem(problem_id, xs=None, delta=0.1, seed=42):
    _, fdata = list(data.items())[problem_id]
    polynomials, indices, edges, objective = process_json(fdata)

    sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(edges[1][idx][0],  rational=True) for idx,poly in polynomials.items()}
    objective = FunctionSympy(objective, indices)

    analyses = {key: s.analysis for key,s in sets.items()}
    residuals = {key: s.residual for key,s in sets.items()}
    order = list(sorted(analyses.keys()))
    A, B = [], order
    # Merge residuals only
    solvefor = torch.tensor([])
    R = EliminateAnalysisMergeResiduals(functions=[residuals[idx] for idx in B])
    P = EliminateAnalysis([analyses[idx] for idx in A], [objective,R])
    objective_idx,residual_idx,equality_idx,inequality_idx = 0,1,None,None
    optim_indices = P.structure[0]
    x0 = perturb(xs, delta, optim_indices, seed=seed)
    # MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
    xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, hobj = generate_optim_functions(P,
        optim_indices, x0, inequality_direction='negative-null', 
        objective=objective_idx, residuals=residual_idx, equalities=equality_idx)
    bnds_problem = [(None, None) for elt in optim_indices]
    xsol, info,storeiter,elapsed_time = ipoptsolvercon(xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, bnds_problem)
    return x0, storeiter,elapsed_time

In [4]:
#x0 = 10*torch.rand(len(indices), dtype=torch.float64)
name = 'results_polyid_{}'.format(str(problem_id))
x0 = load_vals(name, indices)

In [27]:
#seeds = np.random.randint(0,100,10)
seeds = [59, 45, 22, 20, 28, 57,  2, 42, 27, 37]
elapsed_times

[3.226907253265381,
 4.210253477096558,
 4.364189147949219,
 1.0704982280731201,
 6.143305063247681,
 35.00159454345703,
 1.4696159362792969,
 2.564591407775879,
 57.342629194259644,
 6.060227155685425]

In [29]:
elapsed_times

[0.5990188121795654,
 0.4559154510498047,
 0.7027161121368408,
 0.6225996017456055,
 0.5252225399017334,
 0.5669839382171631,
 1.1671006679534912,
 0.709972620010376,
 0.9275097846984863,
 1.0665693283081055]

In [28]:
iters, elapsed_times = [],[]
for seed in seeds:
    xsol, storeiter,elapsed_time = runproblem(0, x0, delta=1, seed=seed)
    iters.append(storeiter)
    elapsed_times.append(elapsed_time)

In [29]:
print_formatted_table([xsol], indices, idxrev)

x_0 x_1 x_2 x_3    x_4   x_5 x_6    x_7 x_8  x_9   x_10  x_11   x_12 x_13 x_14 
1   1   4   -2.267 6.992 1   -0.042 1   2.48 -1.27 -0.45 -2.083 1    -6.5 2.482


In [21]:
results = {str(key):xsol[val].item() for key,val in indices.items()}

In [22]:
name = 'results_polyid_{}.json'.format(str(problem_id))
json_string = json.dumps(results, indent=4)
with open('../applications/data/{}'.format(name), 'w') as file:
    file.write(json_string)
